날짜 변환<br/>
결측 값 행 제거 + 결측 값 많은 열 제거 + 수치 데이터 정규화 작업<br/>
음수 WTI는 새로운 컬럼으로 만들어 처리<br/>
10 단위 1로 수정<br/>
null data 분리

In [11]:
import warnings
warnings.filterwarnings(action="ignore")
import datetime
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [12]:
train_df = pd.read_parquet('../../data/HD_data/train.parquet').drop(columns=['SAMPLE_ID'])
test_df = pd.read_parquet('../../data/HD_data/test.parquet').drop(columns=['SAMPLE_ID'])

In [3]:
train_df.isna().sum()

ARI_CO                     0
ARI_PO                     0
SHIP_TYPE_CATEGORY         0
DIST                       0
ATA                        0
ID                         0
BREADTH                    1
BUILT                      0
DEADWEIGHT                 0
DEPTH                      1
DRAUGHT                    1
GT                         0
LENGTH                     1
SHIPMANAGER                0
FLAG                       0
U_WIND                153486
V_WIND                153486
AIR_TEMPERATURE       154365
BN                    153486
ATA_LT                     0
DUBAI                      0
BRENT                      0
WTI                        0
BDI_ADJ                    0
PORT_SIZE                  0
CI_HOUR                    0
dtype: int64

In [13]:
# datetime 컬럼 처리
train_df['ATA'] = pd.to_datetime(train_df['ATA'])
test_df['ATA'] = pd.to_datetime(test_df['ATA'])

# datetime을 여러 파생 변수로 변환
for df in [train_df, test_df]:
    df['year'] = df['ATA'].dt.year
    df['month'] = df['ATA'].dt.month
    df['day'] = df['ATA'].dt.day
    df['hour'] = df['ATA'].dt.hour
    df['minute'] = df['ATA'].dt.minute
    df['weekday'] = df['ATA'].dt.weekday

# datetime 컬럼 제거
train_df.drop(columns=['ATA'], inplace=True)
test_df.drop(columns=['ATA'], inplace=True)

In [4]:
train_df.iloc[0]

ARI_CO                       SG
ARI_PO                     GIW5
SHIP_TYPE_CATEGORY    Container
DIST                  30.881018
ID                      Z618338
BREADTH                    30.0
BUILT                        24
DEADWEIGHT                24300
DEPTH                      10.0
DRAUGHT                    10.0
GT                        16700
LENGTH                    180.0
SHIPMANAGER              CQSB78
FLAG                     Panama
U_WIND                      NaN
V_WIND                      NaN
AIR_TEMPERATURE             NaN
BN                          NaN
ATA_LT                        5
PORT_SIZE              0.002615
CI_HOUR                    3.45
year                       2018
month                        12
day                          17
hour                         21
minute                       29
weekday                       0
Name: 0, dtype: object

In [14]:
scaler = StandardScaler()
x = train_df[["DIST", "BUILT", "DEADWEIGHT", "GT", "LENGTH", "PORT_SIZE"]]
scaler.fit(x)
train_df[["DIST", "BUILT", "DEADWEIGHT", "GT", "LENGTH", "PORT_SIZE"]] = scaler.transform(x)
test_feautre = test_df[["DIST", "BUILT", "DEADWEIGHT", "GT", "LENGTH", "PORT_SIZE"]]
test_df[["DIST", "BUILT", "DEADWEIGHT", "GT", "LENGTH", "PORT_SIZE"]] = scaler.transform(test_feautre)

In [15]:
for name in ["BREADTH", "DEPTH", "DRAUGHT"]:
    train_df[name] = train_df[name].apply(lambda x: x // 10)
    test_df[name] = test_df[name].apply(lambda x: x // 10)

In [16]:
train_df = train_df.dropna(subset=["BREADTH", "DEPTH", "DRAUGHT", "LENGTH"], axis=0)

In [17]:
len(train_df)

391938

In [18]:
null_idx = train_df[train_df["U_WIND"].isna() | train_df["V_WIND"].isna() | train_df["AIR_TEMPERATURE"].isna() | train_df["BN"].isna()].index
train_null_df = train_df.loc[null_idx]
not_null_idx = train_df[train_df["U_WIND"].notna() & train_df["V_WIND"].notna() & train_df["AIR_TEMPERATURE"].notna() & train_df["BN"].notna()].index
train_not_null_df = train_df.loc[not_null_idx]

In [20]:
print(len(train_null_df))
print(len(train_not_null_df))

164630
227308


In [21]:
len(test_df)

220491

In [22]:
null_idx = test_df[test_df["U_WIND"].isna() | test_df["V_WIND"].isna() | test_df["AIR_TEMPERATURE"].isna() | test_df["BN"].isna()].index
test_null_df = test_df.loc[null_idx]
not_null_idx = test_df[test_df["U_WIND"].notna() & test_df["V_WIND"].notna() & test_df["AIR_TEMPERATURE"].notna() & test_df["BN"].notna()].index
test_not_null_df = test_df.loc[not_null_idx]

In [23]:
print(len(test_null_df))
print(len(test_not_null_df))

92246
128245


In [24]:
train_not_null_df.to_parquet("../../data/HD_data/train_v0_nn.parquet")
train_null_df.to_parquet("../../data/HD_data/train_v0_n.parquet")
test_not_null_df.to_parquet("../../data/HD_data/test_v0_nn.parquet")
test_null_df.to_parquet("../../data/HD_data/test_v0_n.parquet")